In [1]:
import tensorflow as tf
from core.data_sets import train_sentences, train_labels, val_labels, val_sentences, tes
from core.helper_fuctions import calculate_results
from core.helper_functions2 import process_the_model_output
from core.helper_fuctions import create_tensorboard_callback
import tensorflow_hub as hub

In [17]:
sentence_encoder_layer = hub.KerasLayer(
    "https://tfhub.dev/google/universal-sentence-encoder/4", input_shape=[], dtype=tf.string, trainable=False, name="USE")


model_f = tf.keras.Sequential([
    sentence_encoder_layer,
    tf.keras.layers.Dense(160, activation="relu"),
    tf.keras.layers.Dense(80, activation="relu"),
    tf.keras.layers.Dense(1, activation="sigmoid")
])


In [18]:
model_f.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(), metrics=["accuracy"])

model_f_history = model_f.fit(train_sentences, train_labels, epochs=5, validation_data=(
    val_sentences, val_labels), callbacks=[create_tensorboard_callback("model_logs", "final_model")])

process_the_model_output(tf, model_f, val_sentences,
                         val_labels, calculate_results)

Saving TensorBoard log files to: model_logs/final_model/20230103-173753
Epoch 1/5
215/215 [==============================] - 10s 31ms/step - loss: 0.4678 - accuracy: 0.7885 - val_loss: 0.4363 - val_accuracy: 0.8058
Epoch 2/5
215/215 [==============================] - 6s 27ms/step - loss: 0.3925 - accuracy: 0.8318 - val_loss: 0.4272 - val_accuracy: 0.8176
Epoch 3/5
215/215 [==============================] - 6s 27ms/step - loss: 0.3629 - accuracy: 0.8445 - val_loss: 0.4215 - val_accuracy: 0.8228
Epoch 4/5
215/215 [==============================] - 6s 27ms/step - loss: 0.3262 - accuracy: 0.8644 - val_loss: 0.4382 - val_accuracy: 0.8084
Epoch 5/5
24/24 [==============================] - 1s 23ms/step - loss: 0.4741 - accuracy: 0.8123
Model Evaluation result [0.47408753633499146, 0.8123359680175781]
24/24 [==============================] - 1s 22ms/step


{'accuracy': 81.23359580052494,
 'precision': 0.8185848665639733,
 'recall': 0.8123359580052494,
 'f1': 0.809631813400892}

In [4]:
# Possible Improvements
# - Remove the false positives and false negatives from the training data

In [19]:
from core.data_sets import test_df
import random
import pandas as pd

test_sentences = test_df["text"].to_list()


def create_pred_csv(model, sentences, filename="submission_m.csv"):
    # make predictions
    model_pred_probs = model.predict(sentences)
    model_preds = tf.squeeze(tf.round(model_pred_probs))

    model_preds = [int(pred) for pred in model_preds]

    # create a dataframe with two columns, id and target
    ids = test_df["id"].to_list()
    
    df = pd.DataFrame({"id": ids, "target": model_preds})
    df.to_csv(filename, index=False)

create_pred_csv(model_f, test_sentences)



102/102 [==============================] - 2s 22ms/step
